In [13]:

import os
import re
import yaml
import numpy as np
import pandas as pd
import logging

from scipy import stats, signal
from pathlib import Path
from src.lib.class_load import LoadFiles
from src.data.save_models import SAVE_DIR
from src.lib.utils import select_best_model
from src.lib.factory_data import SQLDataSourceFactory, get_data, create_table, set_data
from src.lib.factory_models import ModelContext
from src.lib.factory_prepare_data import (
    DataCleaner,
    DataModel,
    MeanImputation,
    OutliersToIQRMean,
    PrepareDtypeColumns,
    base_dtypes
)
from src.models.DP_model import Modelos
from src.features.features_redis import HandleRedis
from src.features.features_postgres import HandleDBpsql
from src.models.args_data_model import Parameters
from src.data.logs import LOGS_DIR

path_folder = os.path.dirname('/home/bdebian/Documents/Projects/app_prediction/src')
folder_model = Path(path_folder).joinpath("scr/data/save_models")

handler_load = LoadFiles()
handler_redis = HandleRedis()
data_source = HandleDBpsql()
ruta_actual = os.path.dirname('/home/bdebian/Documents/Projects/app_prediction/src')

# =================================================================
#             Configuracion Logger
# =================================================================
# Configura un logger personalizado en lugar de usar el logger raíz
logfile = ruta_actual + "/src/data/config/logging.conf"
logging.config.fileConfig(os.path.join(LOGS_DIR, logfile))
logger = logging.getLogger("predict")
logger.debug("Inciando secuencia de entrenamiento")

# =================================================================
#             Cargar los parametros
# =================================================================
CONFIG_FILE = ruta_actual + "/src/data/config/config.yaml"
with open(CONFIG_FILE, "r", encoding="utf-8") as file:
    parameters = yaml.safe_load(file)

logger.debug("Archivo de configuraciones cargado")
parametros = Parameters(**parameters)

# =================================================================
#             Cargar datos de la fuente de datos
# =================================================================
# Interacion para hacer un cache de los datos en redis
try:
    logger.debug("verficando si existe data cache")
    data = handler_redis.get_cache_data(
        hash_name=parametros.query_template["table"],
        config=parametros.connection_data_source,
    )
    # Condicional para actualizar datos en caso de existan datos en redis
    if data is not None:
        logger.debug("Existe data en cache")

        # Secuencia de codigo para perdir nuevos datos a la base de datos
        date_col_query = parameters["query_template"]["columns"]["0"]
        LAST_DAY = str(data.iloc[-1][0])
        parameters["query_template"]["where"] = f" \"{date_col_query}\" > '{LAST_DAY}'"
        parameters["query_template"]["order"] = "".join(
            ['"' + columna + '"' for columna in [date_col_query]]
        )

        logger.debug("Realizando peticion a la fuente de datos")

        # Extraccion de la nueva data para actualizar
        new = get_data(SQLDataSourceFactory(**parameters))
        logger.debug("Actualizando cache en redis")
        data = handler_redis.set_cache_data(
            hash_name=parametros.query_template["table"],
            old_dataframe=data,
            new_dataframe=new,
            exp_time=parametros.exp_time_cache,
            config=parametros.connection_data_source,
        )
        logger.debug("Actualizacion completa de datos en redis")

    # Verificar que existieran datos en cache
    if data is None:
        logger.debug("No existe cache de datos")

        data = get_data(SQLDataSourceFactory(**parameters))
        logger.debug("Insertando datos de cache en redis")
        data = handler_redis.set_cache_data(
            hash_name=parametros.query_template["table"],
            old_dataframe=data,
            new_dataframe=None,
            exp_time=parametros.exp_time_cache,
            config=parametros.connection_data_source,
        )

except ValueError as error:
    logger.debug("[ERROR] No se puede hacer un cache de la fuente de datos")
    logger.debug(error)
    exit()
# =================================================================
#             Limpieza de datos
# =================================================================
new_types = []

for dtypo in parameters["type_data"].values():

    new_types.append(base_dtypes[dtypo])

# metodo para transformar los tipo de datos
strategy = {int: np.mean, float: np.mean, object: stats.mode}

# Estrategias para imputar los datos faltantes de NA
replace = {
    int: lambda x: int(float(x.replace(",", ""))),
    float: lambda x: float(x.replace(",", "")),
    object: lambda x: x.strip(),
}
# =================================================================

update_dtype_columns = PrepareDtypeColumns(
    replace_dtypes=new_types,
    strategy_imputation=strategy,
    preprocess_function=replace,
    **parameters,
)

# Ejecucion de fabrica para aplicar y ordenar los tipos de datos y los valores
cleaner = DataCleaner()
cleaner.strategy = update_dtype_columns
data_ = cleaner.clean(data)

# Condicion de filtrado para informacion segun los valores
filter_label: str = parameters["filter_data"]["filter_1_feature"]
filter_col: str = parameters["filter_data"]["filter_1_column"]
filter_product = data_.dataframe[filter_col] == filter_label
filter_data = data_.dataframe[filter_product].sort_values(
    by=parameters["filter_data"]["date_column"])

# Seleccion de agrupacion de tiempo
# parameters["filter_data"]["group_frequecy"] = "M"
# parameters["filter_data"]["filter_1_feature"] = filter_label

# # Datos de validacion
# validate_data = filter_data.set_index(time_series_col)["2023-12-01":].reset_index()

# # Datos de entrenamiento
# filter_data = filter_data.set_index(time_series_col)[:"2023-11-30"].reset_index()

outliners = OutliersToIQRMean(**parameters)
cleaner.strategy = outliners
outlines_data = cleaner.clean(filter_data)
# validate_outlines = cleaner.clean(validate_data)
print(outlines_data)
# Filtrado de datos para eliminar valores negativos
filter_values = outlines_data["quantity"] <= 0
outlines_data[filter_values] = 0.1

# =================================================================
#             Filtro pasabajos
# =================================================================
fs = 1 / 24 / 3600  # 1 day in Hz (sampling frequency)

nyquist = fs / 0.5  # 2 # 0.5 times the sampling frequency
cutoff = 0.5  # 0.1 fraction of nyquist frequency, here  it is 5 days
# cutoff=  4.999999999999999  days
b, a = signal.butter(5, cutoff, btype="lowpass")  # low pass filter

dUfilt = signal.filtfilt(b, a, outlines_data["quantity"])
dUfilt = np.array(dUfilt)
dUfilt = dUfilt.transpose()
outlines_data["low_past"] = dUfilt

# =================================================================
#             Preparacion de datos para el modelo
# =================================================================
data_for_model = DataModel(**parameters)
cleaner.strategy = data_for_model
data_ready, scaler_data = cleaner.clean(outlines_data)

# Creacion del dataframe para del filtro pasa bajo para los datos
low_pass_data = outlines_data["low_past"]
low_pass_data = low_pass_data.to_frame()
low_pass_data.rename(columns={"low_past": "quantity"}, inplace=True)
data_ready_lp, scaler_data_lp = cleaner.clean(low_pass_data)

# =================================================================
#            Cargar modelo
# =================================================================
# Rutas de los parametros para predicciones
save_dir = Path(SAVE_DIR).joinpath(parameters["filter_data"]["filter_1_feature"])
models_metrics = save_dir.joinpath("train_metrics").with_suffix(".json").as_posix()

MODE_USED = select_best_model(models_metrics)

scaler_name = save_dir.joinpath("scaler").with_suffix(".pkl").as_posix()
scaler_lp_name = save_dir.joinpath("scaler_lp").with_suffix(".pkl").as_posix()
last_pred = save_dir.joinpath("previus").with_suffix(".json").as_posix()
model_train = save_dir.joinpath(f"model_{MODE_USED}").with_suffix(".pt").as_posix()
parameters_model = save_dir.joinpath(f"parametros_{MODE_USED}").with_suffix(".json").as_posix()

modelo = ModelContext(model_name=MODE_USED,
                      data=data_ready,
                      split=83,
                      covarianze=data_ready_lp,
                      ** parameters
                      )

# Cargar escaler
scaler = handler_load.load_scaler(scaler_name)
scaler_lp = handler_load.load_scaler(scaler_lp_name)

# Cargar modelo para hacer las predicciones
IntModel = Modelos[MODE_USED]
trained_parameters = handler_load.json_to_dict(json_file=parameters_model)[0]
model_update_parameters = IntModel(**trained_parameters)
model_trained = model_update_parameters.load(model_train)

# Se carga el modelo , los datos de predicciones ya se cargaron previamente en ModelContext
pred_series = modelo.predict(
    model=model_trained,
    data=None,
    horizont=parameters["forecast_val"]
)

# Invertir predicciones escaler de entrenamietno
pred_scale = scaler.inverse_transform(pred_series)

# Invertir Predicciones escaler de transformacion de los datos
# pred_scale = scaler_data.inverse_transform(pred_series)

data_frame_predicciones = pred_scale.pd_dataframe()
column_field = list(data_frame_predicciones.columns)
data_frame_predicciones.reset_index(inplace=True)
data_frame_predicciones[parameters["filter_data"]
                        ["predict_column"]].clip(lower=0, inplace=True)

2024-04-04 02:06:19,288 - predict - DEBUG - Inciando secuencia de entrenamiento
2024-04-04 02:06:19,288 - predict - DEBUG - Inciando secuencia de entrenamiento
2024-04-04 02:06:19,290 - predict - DEBUG - Archivo de configuraciones cargado
2024-04-04 02:06:19,290 - predict - DEBUG - Archivo de configuraciones cargado
2024-04-04 02:06:19,291 - predict - DEBUG - verficando si existe data cache
2024-04-04 02:06:19,291 - predict - DEBUG - verficando si existe data cache
2024-04-04 02:06:19,408 - predict - DEBUG - Existe data en cache
2024-04-04 02:06:19,408 - predict - DEBUG - Existe data en cache
2024-04-04 02:06:19,408 - predict - DEBUG - Realizando peticion a la fuente de datos
2024-04-04 02:06:19,408 - predict - DEBUG - Realizando peticion a la fuente de datos
2024-04-04 02:06:19,431 - predict - DEBUG - Actualizando cache en redis
2024-04-04 02:06:19,431 - predict - DEBUG - Actualizando cache en redis
2024-04-04 02:06:19,460 - datasource - DEBUG - Recuperando data existente
2024-04-04 0

/tmp/ipykernel_96929/343251828.py:245: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ["predict_column"]].clip(lower=0, inplace=True)


In [116]:
data_frame_predicciones

component,created_at,quantity
0,2024-04-30,744242.42892


In [2]:
item = filter_label

In [42]:
outlines_data.reset_index(inplace=True)
outlines_data['low_past'] = filter_label
outlines_data.rename({"low_past": filter_col}, axis="columns", inplace=True)

In [81]:
data_.dataframe

,created_at,product,quantity,customer
0,2019-09-26,900006,1000,811007991-8
1,2016-05-28,90000S,2200,900461507-4
2,2021-09-25,13117G,20160,816007113-6
3,2022-04-07,133484BP,3,609172-5
4,2022-07-01,1231210A,450,8404199
...,...,...,...,...
65245,2024-03-11,12112F,2500,901751547-7
65246,2024-03-11,I17806A-26,600,900238561-8
65247,2024-03-11,SCV24,2520,800069933-6
65248,2024-03-11,90006T,200,900714177-4


In [93]:
with open(CONFIG_FILE, "r", encoding="utf-8") as file:
    parameters = yaml.safe_load(file)

logger.debug("Archivo de configuraciones cargado")
parametros = Parameters(**parameters)

2024-04-04 02:49:30,597 - predict - DEBUG - Archivo de configuraciones cargado
2024-04-04 02:49:30,597 - predict - DEBUG - Archivo de configuraciones cargado


In [102]:
type_data_out = {
    "rango": "float",
    "varianza": "float",
    "desviacion_estandar": "float",
    "coeficiente_varianza": "float",
    "quantile_q0": "float",
    "quantile_q1": "float",
    "quantile_q3": "float",
    "quantile_q4": "float",
    "interquantile": "float",
    "desviacion_media_absoluta": "float",
    "init_date": "date",
    "end_date": "date",
    "product": "string",
}

fix_data_dict = {
    "table": "public.datos_originales_metricas",
    "columns": {str(index): key for index, key in enumerate(type_data_out.keys())},
    "order": "index",
    "where": "posicion > 1",
}

parameters["query_template_write"] = fix_data_dict
parameters["type_data_out"] = type_data_out

create_table(SQLDataSourceFactory(**parameters))

In [103]:
create_table(SQLDataSourceFactory(**parameters))

2024-04-04 02:53:56,657 - datasource - DEBUG - Query enviada
2024-04-04 02:53:56,657 - datasource - DEBUG - Query enviada
2024-04-04 02:53:56,658 - datasource - DEBUG - Finalizacion query
2024-04-04 02:53:56,658 - datasource - DEBUG - Finalizacion query


In [105]:
fix_data_dict = {
    "table": "public.datos_originales_metricas",
    "columns": {str(index): key for index, key in enumerate(type_data_out.keys())},
    "order": "index",
    "where": "index > 1",
}

parameters["query_template"] = fix_data_dict
parameters["type_data_out"] = type_data_out
data_last = get_data(SQLDataSourceFactory(**parameters))

In [108]:
data_.dataframe

,created_at,product,quantity,customer,month,year
0,2019-09-26,900006,1000,811007991-8,9,2019
1,2016-05-28,90000S,2200,900461507-4,5,2016
2,2021-09-25,13117G,20160,816007113-6,9,2021
3,2022-04-07,133484BP,3,609172-5,4,2022
4,2022-07-01,1231210A,450,8404199,7,2022
...,...,...,...,...,...,...
65245,2024-03-11,12112F,2500,901751547-7,3,2024
65246,2024-03-11,I17806A-26,600,900238561-8,3,2024
65247,2024-03-11,SCV24,2520,800069933-6,3,2024
65248,2024-03-11,90006T,200,900714177-4,3,2024


In [114]:
LAST_DATE

datetime.date(2023, 12, 31)

In [115]:
data_.dataframe[data_.dataframe['created_at'] > np.datetime64(LAST_DATE)]

,created_at,product,quantity,customer,month,year
52,2024-01-12,90002T,200,811021772-1,1,2024
72,2024-02-02,90000S,200,800010577-2,2,2024
240,2024-01-12,13117F,6600,800227956-3,1,2024
301,2024-02-09,12112F10,675,900198680-3,2,2024
441,2024-01-29,90612,400,24388063-7,1,2024
...,...,...,...,...,...,...
65245,2024-03-11,12112F,2500,901751547-7,3,2024
65246,2024-03-11,I17806A-26,600,900238561-8,3,2024
65247,2024-03-11,SCV24,2520,800069933-6,3,2024
65248,2024-03-11,90006T,200,900714177-4,3,2024


In [110]:
data_.dataframe

,created_at,product,quantity,customer,month,year
0,2019-09-26,900006,1000,811007991-8,9,2019
1,2016-05-28,90000S,2200,900461507-4,5,2016
2,2021-09-25,13117G,20160,816007113-6,9,2021
3,2022-04-07,133484BP,3,609172-5,4,2022
4,2022-07-01,1231210A,450,8404199,7,2022
...,...,...,...,...,...,...
65245,2024-03-11,12112F,2500,901751547-7,3,2024
65246,2024-03-11,I17806A-26,600,900238561-8,3,2024
65247,2024-03-11,SCV24,2520,800069933-6,3,2024
65248,2024-03-11,90006T,200,900714177-4,3,2024


In [125]:
df = data_.dataframe

# Convertir la columna 'created_at' a tipo datetime
df['created_at'] = pd.to_datetime(df['created_at'])

# Agregar columnas de mes y año
df['month'] = df['created_at'].dt.month
df['year'] = df['created_at'].dt.year

# Calcular estadísticas por mes
monthly_stats = df.groupby(['year', 'month']).agg(
    rango=('quantity', lambda x: x.max() - x.min()),
    # varianza=('quantity', 'var'),
    desviacion_estandar=('quantity', 'std'),
    coeficiente_varianza=('quantity', lambda x: x.std() / x.mean()),
    quantile_q0=('quantity', lambda x: x.quantile(0)),
    quantile_q1=('quantity', lambda x: x.quantile(0.25)),
    quantile_q3=('quantity', lambda x: x.quantile(0.75)),
    quantile_q4=('quantity', lambda x: x.quantile(1)),
    interquantile=('quantity', lambda x: x.quantile(0.75) - x.quantile(0.25)),
    desviacion_media_absoluta=('quantity', lambda x: (x - x.mean()).abs().mean())
).reset_index()

# Convertir año y mes a fecha inicial y final del mes
monthly_stats['init_date'] = pd.to_datetime(monthly_stats[['year', 'month']].assign(day=1))
monthly_stats['end_date'] = pd.to_datetime(monthly_stats[['year', 'month']].assign(day=pd.DatetimeIndex(pd.to_datetime(monthly_stats['init_date'])).days_in_month))

# Eliminar columnas de año y mes
monthly_stats.drop(columns=['year', 'month'], inplace=True)

# Agregar la columna 'product' al DataFrame resultante
monthly_stats['product'] = df['product'].iloc[0]

# Reordenar las columnas según el tipo de datos
monthly_stats = monthly_stats[['rango',
                                #    'varianza',
                                   'desviacion_estandar',
                                   'coeficiente_varianza',
                                   'quantile_q0',
                                   'quantile_q1',
                                   'quantile_q3',
                                   'quantile_q4',
                                   'interquantile',
                                   'desviacion_media_absoluta',
                                   'init_date',
                                   'end_date',
                                   'product']]




In [127]:
# monthly_stats['desviacion_estandar'].astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [126]:
display(monthly_stats)

,rango,desviacion_estandar,coeficiente_varianza,quantile_q0,quantile_q1,quantile_q3,quantile_q4,interquantile,desviacion_media_absoluta,init_date,end_date,product
0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,2002-12-01,2002-12-31,900006
1,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,2003-02-01,2003-02-28,900006
2,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,2003-09-01,2003-09-30,900006
3,0,NaN,NaN,2.0,2.0,2.0,2.0,0.0,0.000000,2004-02-01,2004-02-29,900006
4,591,208.842312,0.773808,9.0,120.0,500.0,600.0,380.0,175.629630,2004-03-01,2004-03-31,900006
...,...,...,...,...,...,...,...,...,...,...,...,...
159,460050,15887.462139,3.775927,-160000.0,400.0,3000.0,300050.0,2600.0,5537.202621,2023-11-01,2023-11-30,900006
160,493750,18896.005950,3.301843,-5000.0,450.0,5000.0,488750.0,4550.0,6692.376177,2023-12-01,2023-12-31,900006
161,510000,21507.905924,3.952478,-160000.0,400.0,4350.0,350000.0,3950.0,8132.658555,2024-01-01,2024-01-31,900006
162,319600,13159.084471,3.073951,-32600.0,324.0,3000.0,287000.0,2676.0,5388.584681,2024-02-01,2024-02-29,900006


In [51]:
from pathlib import Path
import numpy as np
def load_path_names(path_dir: str, extensions: list[str]) -> tuple[list[str], list[str]]:
    """
    Lee los archivos de una carpeta y retorna una lista con la ruta completa del archivo
    y otra con solo el nombre del archivo

    @path_dir:
        Info:Especificacion de la ruta de un directorio para leer archivos
        Dtype:String

    @extenciones:
        Info: Lista de string separda por comas con las extenciones que se desean buscar en la carpeta
        Dtype:Lista Strings ['.tiff','.jpg','.jpge','.tif','.png]

    @sample:
        Info: Retorna una lista con la ruta completa del las imagenes
        Dtype:String

    @names:
        Info: Retorna una lista unicamente con los nombres de los archivos
        Dtype: String
    """

    extensions = ["." + i for i in extensions]
    extensions = extensions + [i.upper() for i in extensions]
    extensions = sorted(set(extensions))

    data_path = Path(path_dir)

    if data_path.is_dir():
        sample = []
        names = []

        for ext in extensions:
            t_file = "**/*" + ext
            if len(list(data_path.glob(t_file))) == 0:
                continue
            else:
                sample.extend(list(map(str, data_path.glob(t_file))))

        sample = sorted(set(sample))

        if not sample:
            print(
                f"\n[INFO] No hay imagenes con estas extensiones:\n\t{extensions}\n\tEn la ruta:\n\t{path_dir}\n"
            )
            return [], []

        else:

            if len(sample) != 1:
                names = [Path(p).parts[-1] for p in sample]
                return sorted(set(sample)), sorted(set(names))
            else:
                return sorted(set([sample[0]])), sorted(set([Path(sample[0]).parts[-1]])
)
    else:
        return print("[INFO], la ruta no existe, favor revisar el directorio")

In [52]:
salida1 = load_path_names(path_dir='/home/bdebian/Documents/Projects/app_prediction/',
                extensions = ['py']
                )[1]

In [53]:
salida1

['DP_model.py',
 '__init__.py',
 'alerts_data.py',
 'alerts_predict.py',
 'args_data_model.py',
 'class_create_tree_folders.py',
 'class_load.py',
 'factory_data.py',
 'factory_models.py',
 'factory_prepare_data.py',
 'features_fix_data.py',
 'features_plaintext.py',
 'features_postgres.py',
 'features_redis.py',
 'features_sqlserver.py',
 'filter_data.py',
 'pandas_utils.py',
 'predict.py',
 'run_predict.py',
 'run_train.py',
 'train.py',
 'utils.py']

In [22]:
def load_path_names( path_dir: str, extenciones: list):
    """
    Lee los archivos de una carpeta y retorna una lista con la ruta completa del archivo
    y otra con solo el nombre del archivo

    @path_dir:
        Info:Especificacion de la ruta de un directorio para leer archivos
        Dtype:String

    @extenciones:
        Info: Lista de string separda por comas con las extenciones que se desean buscar en la carpeta
        Dtype:Lista Strings ['.tiff','.jpg','.jpge','.tif','.png'load_image_path]

    @sample:
        Info: Retorna una lista con la ruta completa del las imagenes
        Dtype:String

    @names:
        Info: Retorna una lista unicamente con los nombres de los archivos
        Dtype: String
    """

    extenciones = ["." + i for i in extenciones]
    extenciones = extenciones + [i.upper() for i in extenciones]
    extenciones = sorted(set(extenciones))

    data_path = Path(path_dir)

    if data_path.is_dir():
        sample = []
        names = []

        for i in extenciones:
            t_file = "**/*" + i
            if len(list(data_path.glob(t_file))) == 0:
                continue
            else:
                sample.append(list(data_path.glob(t_file)))

        def flatten(lista):
            return [item for sublist in lista for item in sublist]

        sample = flatten(sample)
        sample = sorted(set(sample))

        if not sample:
            print(
                f"\n[INFO] No hay imagenes con estas extenciones:\n\t{extenciones}\n\tEn la ruta:\n\t{path_dir}\n"
            )
            return [], []

        else:

            if len(sample) != 1:

                for name in sample:
                    names.append(name.parts[-1])
                sample = np.array([str(i) for i in sample])
                return sorted(set(sample)), sorted(set(names))
            else:
                return str(sample[0]), str(sample[0].parts[-1])

    else:
        return print("[INFO], la ruta no existe, favor revisar el directorio")

In [23]:
load_path_names(path_dir='/home/bdebian/Documents/Projects/app_prediction/',
                extenciones = ['py']
                )[1]

['DP_model.py',
 '__init__.py',
 'alerts_data.py',
 'alerts_predict.py',
 'args_data_model.py',
 'class_create_tree_folders.py',
 'class_load.py',
 'factory_data.py',
 'factory_models.py',
 'factory_prepare_data.py',
 'features_fix_data.py',
 'features_plaintext.py',
 'features_postgres.py',
 'features_redis.py',
 'features_sqlserver.py',
 'filter_data.py',
 'pandas_utils.py',
 'predict.py',
 'run_predict.py',
 'run_train.py',
 'train.py',
 'utils.py']